# 0. Imports

In [19]:
import pandas as pd
import numpy as np

from sklearn import model_selection as ms
from sklearn import metrics as mt
from sklearn import ensemble as en
from sklearn import dummy as dm

In [20]:
df_raw = pd.read_csv( 'dataset/train_users_2.csv')
df_raw.head()

,id,date_account_created,timestamp_first_active,date_first_booking,gender,age,signup_method,signup_flow,language,affiliate_channel,affiliate_provider,first_affiliate_tracked,signup_app,first_device_type,first_browser,country_destination
0,gxn3p5htnn,2010-06-28,20090319043255,NaN,-unknown-,NaN,facebook,0,en,direct,direct,untracked,Web,Mac Desktop,Chrome,NDF
1,820tgsjxq7,2011-05-25,20090523174809,NaN,MALE,38.0,facebook,0,en,seo,google,untracked,Web,Mac Desktop,Chrome,NDF
2,4ft3gnwmtx,2010-09-28,20090609231247,2010-08-02,FEMALE,56.0,basic,3,en,direct,direct,untracked,Web,Windows Desktop,IE,US
3,bjjt8pjhuk,2011-12-05,20091031060129,2012-09-08,FEMALE,42.0,facebook,0,en,direct,direct,untracked,Web,Mac Desktop,Firefox,other
4,87mebub9p4,2010-09-14,20091208061105,2010-02-18,-unknown-,41.0,basic,0,en,direct,direct,untracked,Web,Mac Desktop,Chrome,US


# 1.0. Data Descriptive

## 1.1. Train Test Split

In [21]:
# X_train and y_train
X_train = df_raw.drop('country_destination', axis=1)
y_train = df_raw['country_destination']

# split data into train and test
X_train, X_test, y_train, y_test = ms.train_test_split(
    X_train, 
    y_train,
    stratify=y_train,
    
    test_size=0.2 )


df1 = pd.concat( [X_train, y_train], axis=1 )
df1.head()

,id,date_account_created,timestamp_first_active,date_first_booking,gender,age,signup_method,signup_flow,language,affiliate_channel,affiliate_provider,first_affiliate_tracked,signup_app,first_device_type,first_browser,country_destination
21198,zrq37tx8lt,2012-04-13,20120413024112,NaN,FEMALE,60.0,facebook,0,en,direct,direct,untracked,Web,Mac Desktop,Safari,NDF
45464,b7khgmz26p,2012-10-19,20121019192933,2012-10-19,-unknown-,NaN,basic,0,en,direct,direct,linked,Web,Mac Desktop,Safari,US
99583,3wx6pg1y3j,2013-08-15,20130815164518,NaN,FEMALE,33.0,basic,0,en,direct,direct,linked,Web,Windows Desktop,Firefox,NDF
99200,j81r3khp7s,2013-08-14,20130814045037,2013-08-15,-unknown-,NaN,basic,0,en,direct,direct,linked,Web,Mac Desktop,Firefox,US
61040,ejjrv1rcwo,2013-02-18,20130218204050,NaN,-unknown-,NaN,basic,0,en,sem-non-brand,google,omg,Web,Windows Desktop,Chrome,NDF


## 1.2. Data Descriptive

In [22]:
df1.info()

<class 'pandas.core.frame.DataFrame'>
Index: 170760 entries, 21198 to 79510
Data columns (total 16 columns):
 #   Column                   Non-Null Count   Dtype  
---  ------                   --------------   -----  
 0   id                       170760 non-null  object 
 1   date_account_created     170760 non-null  object 
 2   timestamp_first_active   170760 non-null  int64  
 3   date_first_booking       71126 non-null   object 
 4   gender                   170760 non-null  object 
 5   age                      100262 non-null  float64
 6   signup_method            170760 non-null  object 
 7   signup_flow              170760 non-null  int64  
 8   language                 170760 non-null  object 
 9   affiliate_channel        170760 non-null  object 
 10  affiliate_provider       170760 non-null  object 
 11  first_affiliate_tracked  165889 non-null  object 
 12  signup_app               170760 non-null  object 
 13  first_device_type        170760 non-null  object 
 14  first_

## 1.3. Missing Values

In [23]:
df1.isna().sum() / df1.shape[0]

id                         0.000000
date_account_created       0.000000
timestamp_first_active     0.000000
date_first_booking         0.583474
gender                     0.000000
age                        0.412848
signup_method              0.000000
signup_flow                0.000000
language                   0.000000
affiliate_channel          0.000000
affiliate_provider         0.000000
first_affiliate_tracked    0.028525
signup_app                 0.000000
first_device_type          0.000000
first_browser              0.000000
country_destination        0.000000
dtype: float64

## 1.4. Numerical Descriptive

In [24]:
# change dtypes
df1['date_account_created'] = pd.to_datetime( df1['date_account_created'] )
df1['date_first_booking'] = pd.to_datetime( df1['date_first_booking'] )
df1['timestamp_first_active'] = pd.to_datetime( df1['timestamp_first_active'], format="%Y%m%d%H%M%S" ) 

num_cols = df1.select_dtypes( include=['int64', 'float64'] )
cat_cols = df1.select_dtypes( exclude=['int64', 'float64', 'datetime64[ns]'] )

In [25]:
# central tendency - mean, median
ct1 = pd.DataFrame( num_cols.apply( np.mean ) )
ct2 = pd.DataFrame( num_cols.apply( np.median ) )

# dispersion - std, min, max, range, skew, kurtosis
d1 = pd.DataFrame( num_cols.apply( np.std ) )
d2 = pd.DataFrame( num_cols.apply( np.min ) )
d3 = pd.DataFrame( num_cols.apply( np.max ) )
d4 = pd.DataFrame( num_cols.apply( lambda x: x.max() - x.min() ) )
d5 = pd.DataFrame( num_cols.apply( lambda x: x.skew() ) )
d6 = pd.DataFrame( num_cols.apply( lambda x: x.kurt() ) )

# concatenate
m = pd.concat( [d2, d3, d4, ct1, ct2, d1, d5, d6], axis=1 ).reset_index()
m.columns = ['attributes', 'min', 'max', 'range', 'mean', 'median', 'std', 'skew', 'kurtosis']

m

,attributes,min,max,range,mean,median,std,skew,kurtosis
0,age,1.0,2014.0,2013.0,49.761116,NaN,156.328845,12.368102,152.203313
1,signup_flow,0.0,25.0,25.0,3.271018,0.0,7.640989,2.232232,3.315250


## 1.5. Categorical Descriptive

### 1.5.1 Genre

In [ ]:
pt = pd.pivot_table( data=df1, index='country_destination', columns='gender', values='id', aggfunc='count' )
pt_row_pct = pt.div(pt.sum(axis=1), axis=0).mul(100).round(1)
pt_row_pct


gender,-unknown-,FEMALE,MALE,OTHER
country_destination,,,,
AU,25.1,39.7,35.0,0.2
CA,35.5,29.9,34.2,0.4
DE,26.4,33.7,39.6,0.4
ES,32.1,37.1,30.5,0.2
FR,33.9,39.1,26.7,0.2
GB,32.5,38.3,29.0,0.1
IT,37.2,38.4,24.3,0.2
NDF,53.7,24.9,21.4,0.1
NL,28.9,32.8,37.9,0.5


### 1.5.2 Signup method

In [35]:
pt = pd.pivot_table( data=df1, index='country_destination', columns='signup_method', values='id', aggfunc='count' )
pt_row_pct = pt.div(pt.sum(axis=1), axis=0).mul(100).round(1)
pt_row_pct

language,ca,cs,da,de,el,en,es,fi,fr,hr,...,ko,nl,no,pl,pt,ru,sv,th,tr,zh
country_destination,,,,,,,,,,,,,,,,,,,,,
AU,NaN,NaN,NaN,0.7,NaN,98.4,NaN,NaN,0.5,NaN,...,0.2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.2
CA,NaN,NaN,NaN,0.1,NaN,98.7,NaN,NaN,0.5,NaN,...,0.2,NaN,NaN,NaN,NaN,0.1,NaN,NaN,NaN,0.2
DE,NaN,NaN,NaN,2.1,NaN,95.5,0.7,NaN,0.9,NaN,...,NaN,0.1,NaN,NaN,NaN,0.1,NaN,NaN,NaN,0.2
ES,NaN,0.1,0.1,0.3,0.1,97.2,0.9,NaN,0.4,NaN,...,0.1,NaN,NaN,0.1,0.1,0.3,0.1,NaN,NaN,0.1
FR,NaN,NaN,NaN,0.2,0.0,97.0,0.3,0.0,1.6,NaN,...,0.3,0.0,NaN,0.0,NaN,0.1,NaN,NaN,NaN,0.2
GB,NaN,NaN,NaN,0.3,NaN,98.3,0.3,NaN,0.5,NaN,...,0.1,NaN,0.1,NaN,NaN,0.1,NaN,NaN,NaN,0.2
IT,NaN,NaN,0.0,0.2,NaN,97.4,0.3,NaN,0.4,NaN,...,0.4,0.0,NaN,0.0,NaN,0.2,NaN,NaN,0.0,0.2
NDF,0.0,0.0,0.0,0.4,0.0,96.0,0.5,0.0,0.6,0.0,...,0.4,0.0,0.0,0.0,0.2,0.2,0.1,0.0,0.0,0.9
NL,NaN,NaN,NaN,0.3,NaN,97.7,0.2,NaN,0.7,NaN,...,0.3,0.3,NaN,NaN,0.2,NaN,0.2,NaN,NaN,NaN


### 1.5.1 Affiliate Channel

In [ ]:
pt = pd.pivot_table( data=df1, index='country_destination', columns='affiliate_channel', values='id', aggfunc='count' )
pt_row_pct = pt.div(pt.sum(axis=1), axis=0).mul(100).round(1)
pt_row_pct

affiliate_channel,api,content,direct,other,remarketing,sem-brand,sem-non-brand,seo
country_destination,,,,,,,,
AU,3.5,1.2,67.5,1.6,NaN,12.1,11.1,3.0
CA,1.8,0.8,67.3,4.9,0.4,10.8,10.1,3.9
DE,2.0,0.4,66.0,3.4,0.4,12.8,10.7,4.4
ES,2.4,1.1,67.9,1.5,0.9,12.6,8.5,5.1
FR,1.8,1.2,65.5,2.3,0.7,13.1,10.5,4.9
GB,2.6,0.2,67.1,2.2,0.6,14.1,8.6,4.5
IT,1.7,0.7,65.4,2.3,0.5,16.5,8.8,4.0
NDF,4.3,2.7,63.0,4.3,0.6,11.9,9.4,3.8
NL,2.0,1.0,66.9,1.3,0.3,13.6,10.3,4.6


### 1.5.1 Affiliate Provider

In [37]:
pt = pd.pivot_table( data=df1, index='country_destination', columns='affiliate_provider', values='id', aggfunc='count' )
pt_row_pct = pt.div(pt.sum(axis=1), axis=0).mul(100).round(1)
pt_row_pct

affiliate_provider,baidu,bing,craigslist,daum,direct,email-marketing,facebook,facebook-open-graph,google,gsp,meetup,naver,other,padmapper,vast,wayn,yahoo,yandex
country_destination,,,,,,,,,,,,,,,,,,
AU,NaN,1.6,0.7,NaN,67.3,NaN,0.2,NaN,25.5,NaN,NaN,NaN,4.6,NaN,NaN,NaN,NaN,NaN
CA,NaN,0.8,2.9,NaN,67.2,NaN,0.9,0.2,23.4,NaN,NaN,NaN,4.0,0.4,0.3,NaN,NaN,NaN
DE,NaN,1.8,1.5,NaN,66.0,NaN,1.6,0.2,24.7,NaN,NaN,NaN,3.5,0.1,0.4,NaN,0.1,NaN
ES,NaN,1.0,0.4,NaN,67.5,NaN,1.2,0.1,25.1,NaN,NaN,NaN,4.0,0.1,0.3,NaN,0.2,NaN
FR,0.0,1.0,1.2,NaN,65.3,NaN,1.0,0.2,27.5,0.0,0.0,0.0,3.0,0.1,0.3,NaN,0.3,NaN
GB,NaN,0.9,0.9,NaN,67.0,0.1,1.2,0.2,25.6,NaN,0.1,NaN,3.8,0.2,0.1,NaN,0.2,NaN
IT,NaN,1.4,0.9,NaN,65.3,0.0,0.9,0.1,27.6,0.0,0.1,0.0,3.0,0.0,0.2,NaN,0.1,NaN
NDF,0.0,1.2,1.5,NaN,62.9,0.1,1.2,0.3,24.7,0.3,0.2,0.0,6.3,0.4,0.5,0.0,0.3,0.0
NL,NaN,0.8,0.3,NaN,66.4,NaN,1.5,0.3,27.2,NaN,0.2,NaN,3.1,NaN,0.2,NaN,NaN,NaN


### 1.5.1 First Affiliate Tracker

In [38]:
pt = pd.pivot_table( data=df1, index='country_destination', columns='first_affiliate_tracked', values='id', aggfunc='count' )
pt_row_pct = pt.div(pt.sum(axis=1), axis=0).mul(100).round(1)
pt_row_pct

first_affiliate_tracked,linked,local ops,marketing,omg,product,tracked-other,untracked
country_destination,,,,,,,
AU,20.3,0.2,NaN,22.9,0.7,0.9,54.8
CA,21.7,NaN,0.1,18.3,0.6,2.7,56.6
DE,21.9,NaN,NaN,19.2,0.2,2.5,56.1
ES,24.2,NaN,NaN,19.5,0.4,1.5,54.4
FR,21.9,NaN,0.1,20.9,0.6,1.8,54.6
GB,21.6,NaN,0.1,22.0,0.3,2.3,53.8
IT,19.5,NaN,0.0,22.4,1.0,2.0,55.1
NDF,21.8,0.0,0.1,22.9,0.8,3.2,51.2
NL,21.0,NaN,NaN,21.9,1.2,1.3,54.6


### 1.5.1. Signup App

In [40]:
pt = pd.pivot_table( data=df1, index='country_destination', columns='signup_app', values='id', aggfunc='count' )
pt_row_pct = pt.div(pt.sum(axis=1), axis=0).mul(100).round(1)
pt_row_pct

signup_app,Android,Moweb,Web,iOS
country_destination,,,,
AU,0.7,2.8,91.0,5.6
CA,1.1,2.2,92.6,4.0
DE,0.8,1.2,93.6,4.4
ES,1.2,1.4,92.8,4.5
FR,0.4,1.2,94.2,4.2
GB,0.8,1.6,92.7,4.9
IT,0.6,1.1,93.0,5.2
NDF,3.4,3.3,82.4,10.9
NL,0.8,2.0,93.1,4.1


### 1.5.1. First Device Type

In [41]:
pt = pd.pivot_table( data=df1, index='country_destination', columns='first_device_type', values='id', aggfunc='count' )
pt_row_pct = pt.div(pt.sum(axis=1), axis=0).mul(100).round(1)
pt_row_pct

first_device_type,Android Phone,Android Tablet,Desktop (Other),Mac Desktop,Other/Unknown,SmartPhone (Other),Windows Desktop,iPad,iPhone
country_destination,,,,,,,,,
AU,0.2,0.5,0.7,48.3,3.0,NaN,32.9,7.0,7.4
CA,1.0,0.9,1.3,47.8,3.6,NaN,36.2,5.2,4.1
DE,0.4,0.5,1.3,52.9,2.4,NaN,31.0,5.7,6.0
ES,0.6,0.6,0.4,48.6,2.7,NaN,34.7,6.8,5.6
FR,0.3,0.5,0.5,51.4,3.3,0.0,31.7,7.0,5.1
GB,0.1,0.2,0.2,51.3,2.4,NaN,33.1,7.0,5.6
IT,0.5,0.6,0.5,48.0,1.9,0.0,34.3,8.2,6.0
NDF,1.7,0.7,0.5,38.0,6.3,0.0,34.2,7.1,11.5
NL,0.8,0.7,0.8,52.3,2.8,NaN,28.7,8.9,5.1


### 1.5.1. First Browser

In [43]:
pt = pd.pivot_table( data=df1, index='country_destination', columns='first_browser', values='id', aggfunc='count' )
pt_row_pct = pt.div(pt.sum(axis=1), axis=0).mul(100).round(1)
pt_row_pct

first_browser,-unknown-,AOL Explorer,Android Browser,Apple Mail,Arora,Avant Browser,BlackBerry Browser,Camino,Chrome,Chrome Mobile,...,SeaMonkey,Silk,SiteKiosk,SlimBrowser,Sogou Explorer,Stainless,TenFourFox,TheWorld Browser,Yandex.Browser,wOSBrowser
country_destination,,,,,,,,,,,,,,,,,,,,,
AU,7.2,NaN,0.2,NaN,NaN,NaN,0.2,NaN,30.6,0.5,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
CA,7.0,0.1,0.2,NaN,NaN,NaN,NaN,NaN,34.4,0.4,...,0.1,0.1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
DE,7.4,NaN,0.2,0.1,NaN,NaN,NaN,NaN,33.7,0.6,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
ES,7.0,0.1,0.2,NaN,NaN,NaN,NaN,NaN,34.0,0.2,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
FR,7.0,0.1,0.2,NaN,NaN,NaN,0.0,NaN,31.0,0.2,...,NaN,0.0,0.0,NaN,NaN,NaN,NaN,0.0,NaN,NaN
GB,6.8,0.1,0.1,NaN,NaN,NaN,NaN,NaN,32.4,0.5,...,NaN,0.1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
IT,6.7,0.1,0.2,NaN,NaN,NaN,0.1,NaN,31.9,0.4,...,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
NDF,15.8,0.1,0.5,0.0,0.0,0.0,0.0,0.0,27.3,0.7,...,0.0,0.1,0.0,0.0,0.0,NaN,0.0,NaN,0.0,0.0
NL,7.2,NaN,0.3,NaN,NaN,NaN,NaN,NaN,33.6,0.8,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


# 2.0. Features Engineering

# 3.0. Data Filtering

In [ ]:
cols_to_drop = [
    'id',
    'date_account_created',
    'date_first_booking',
    'timestamp_first_active'
]

# drop columns
df1 = df1.drop( cols_to_drop, axis=1 )
df1.head()

# 4.0. Exploratory Data Analysis

In [ ]:
# grafico de barras mostrando a distribuicao de usuarios por pais
df1['country_destination'].value_counts().plot( kind='bar' )

# 5.0. Data Preparation

In [ ]:
X_tr = df1.drop('country_destination', axis=1)
y_tr = df1['country_destination']

## 5.1. Normalization

## 5.2. Scaling

## 5.3. Encoder

In [ ]:
# Frequency Encoder
for col in X_tr.select_dtypes( include=['object'] ).columns:
    freq = X_tr[col].value_counts().to_dict()
    X_tr[col] = X_tr[col].map( freq )

In [ ]:
X_tr.head()

## 5.4. Transformation

# 6.0. Feature Selection

# 7.0. Machine Learning Modeling

## 7.1. Random Classifier

In [ ]:
# Training Dummy Classifier

# definition
dummy_model = dm.DummyClassifier( strategy='most_frequent' )

# train
dummy_model.fit( X_tr, y_tr )

# prediction
y_pred_tr = dummy_model.predict( X_tr )

# performance
dummy_acc_tr = mt.accuracy_score( y_tr, y_pred_tr ) 
dummy_f1_score = mt.f1_score( y_tr, y_pred_tr, average='macro' )

print( 'Accuracy: ', dummy_acc_tr )
print( 'F1 Score: ', dummy_f1_score )

# 7.1. Random Forest Classifier

In [ ]:
# model definition
model_rf = en.RandomForestClassifier( n_estimators=100, random_state=42, n_jobs=-1 )

# training
model_rf.fit( X_tr, y_tr )

# prediction
y_pred_tr = model_rf.predict( X_tr )

# performance
model_rf_acc_tr = mt.accuracy_score( y_tr, y_pred_tr ) 
model_rf_f1_score = mt.f1_score( y_tr, y_pred_tr, average='macro' )

print( 'Accuracy: ', model_rf_acc_tr )
print( 'F1 Score: ', model_rf_f1_score )

# 8.0. Hyperparameter Fine Tuning

# 9.0. Performance Metrics

# 10.0. Deploy to Production